# Input data and temperature sensitivity

A continuation of the simple sensitivity study we did in 
[Sensitivity studies using COSIPY](sensitivity_study.ipynb) is to investigate how changing the surface temperature, or any other input variable, affects the calculation of the surface mass balance. COSIPY reads the meteorological input data from a **netcdf** file during the run, i.e. during each time step the model reads the meteorological data from the corresponding time step in the input file. The input file can be either "1D" for a point simulation or "2D" for a distributed simulation. The input file can be based on either observed or modeled data.

In order to conduct the temperature sensitivity study we first need to do the following:
- Create the input file from observations.
- Create copies of the input netcdf with different temperature biases.
- Adapt the `NAMELIST` instances to point to the biased files. 

With this done we can run the simulations. But first let's take a look at the input data for Zhadang glacier, which ships with COSPIY.

**The standard imports**

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')

In [ ]:
from cosipy.cpkernel.cosipy_core import cosipy_core
from cosipy.cpkernel.io import IOClass
# cfg gives us the NAMELIST
import cfg
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from IPython.display import set_matplotlib_formats
# Nice plots
set_matplotlib_formats('svg')
# Have to tell matplotlib to plot inline
%matplotlib inline

The NAMELIST already contains the path to the input data, let us open it with `xarray`

In [ ]:
import xarray as xr

In [ ]:
# Path to data
cfg.NAMELIST['data_path']

In [ ]:
# The path of the file
cfg.NAMELIST['input_netcdf']

Open the file with xarray:

In [ ]:
# Use the namelist for path to data and input netcdf. Have to add
# input in between.
input_path = cfg.NAMELIST['data_path'] + 'input/' +\
             cfg.NAMELIST['input_netcdf']
with xr.open_dataset(input_path) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

As you can see, this file contain variables such as the 2-meter temperature (T2), relative humidity at 2 meters (RH2) and the cloud cover (N) for one single point. The time coordinate spans between 2009-01-01 00:00:00 and 2009-01-31 22:00:00 on a hourly resolution.


<div class="alert alert-warning">
    <b>Question: Can you figure out what the variable G is and what unit it has?</b>
</div>
<div class="alert alert-success">
<details>
    <summary><b>Click me for a hint!</b></summary>
    Try pressing the document symbol to the right in the output above, this shows some extra information about the variable.
    </details>
</div>

We can quickly take a look at one of the variables

In [ ]:
ds.T2.plot();

## Creating the input files

In this case COSIPY comes packaged with the processed data which can directly be used to drive a simulation. In a more realistic scenario however, you probably want drive COSIPY with your own data on another glacier than the Zhadang glacier. The `aws2cosipy` module is provided by COSIPY to aid the processing of .csv-files from weather stations into input files which can be used by COSIPY. 

In [ ]:
# We import the create_1D_input function to process a single point
from utilities.aws2cosipy.aws2cosipy import create_1D_input

The `create_1D_input` has five arguments: the csv file to process, the name of the resulting input file, the name of a static file describing the altitude, slope and aspect of the point, and the start and end date.

The .csv file is located in the same directory as the input file we looked at earlier. It is called `Zhadang_ERA5_2009_2018.csv`. We can process it

In [ ]:
# Path to the file
data_folder = cfg.NAMELIST['data_path']
file = data_folder + 'input/Zhadang/Zhadang_ERA5_2009_2018.csv'

In [ ]:
# Static file
static_file = data_folder + 'static/Zhadang_static.nc'

In [ ]:
# Output file
output_name = data_folder + 'input/Zhadang/Zhadang_ERA5_2009.nc'

In [ ]:
# Define time start and end.
start_date = '20090101'
end_date = '20090131'

In [ ]:
# Call the function, this takes some time.
create_1D_input(file, output_name, static_file, start_date, end_date)

Open the file to see that it worked

In [ ]:
with xr.open_dataset(output_name) as ds:
    ds = ds.isel(time=slice(0, -1)).load()

In [ ]:
ds

This basically recreated the file that we already had, but feel free to try it on another time period. The .csv-file contains data from January 2000 until early January 2018.

## Creating data for temperature bias experiments

A temperature bias experiment is essentially a sensitivity study. It explores the effects of changing the temperature by $n$ degrees over the whole time period, let's say we want to know how the glacier responds to a climate which is  two degrees C warmer. However this approach is quite rough since it doesn't take other changes that might come with a warmer climate into account.

Setting this up with COSIPY follows a similar approach as described in the [Sensitivity studies with COSIPY](sensitivity_study.ipynb), with the additional step that we have to create one input file for every experiment. Since reading the .csv-file is rather slow, we will start from the file we already have and create copies from it.

In [ ]:
# We already have the xarray object in memory, hence we can creaty copies
ds_up = ds.copy()
ds_dn = ds.copy()

We start with a bias of 2 °C

In [ ]:
bias = 2

In [ ]:
# We basically update the variable with the old variable +/- the bias
ds_up['T2'] = ds_up['T2'] + bias
ds_dn['T2'] = ds_dn['T2'] - bias

**Plot it to make sure it worked**

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ds['T2'].plot(ax=ax, label='Default')
ds_up['T2'].plot(ax=ax, label='+2')
ds_dn['T2'].plot(ax=ax, label='-2')
plt.legend();

It is not the best looking plot, but it serves its purpose to show that the temperatures are slightly offset from each other.

<div class="alert alert-info">
    <b>Important!</b> We have to save the new datasets!
</div>

In [ ]:
# Declare the paths
fpth_up = data_folder + 'input/Zhadang/Zhadang_ERA5_2009_up.nc'
fpth_dn = data_folder + 'input/Zhadang/Zhadang_ERA5_2009_dn.nc'

In [ ]:
# Save the datasets
ds_up.to_netcdf(fpth_up)
ds_dn.to_netcdf(fpth_dn)

## Setting up the experiments

We now have the input files for our planned runs. The runs can be set up to run similarly to our previous albedo experiments, but this time we change the `input_netcdf` instead.

In [ ]:
cfg.NAMELIST['input_netcdf']

In [ ]:
# We create a copy of the namelist so we don't change the original.
# Default
NAMELIST_def = cfg.NAMELIST.copy()
# Up 10%
NAMELIST_up = cfg.NAMELIST.copy()
# Set it to our new file
NAMELIST_up['input_netcdf'] = 'Zhadang/Zhadang_ERA5_2009_up.nc'
# Down 10%
NAMELIST_dn =cfg.NAMELIST.copy()
NAMELIST_dn['input_netcdf'] = 'Zhadang/Zhadang_ERA5_2009_dn.nc'

In [ ]:
# Default
IO_def = IOClass(NAMELIST_def) 
# Down
IO_dn = IOClass(NAMELIST_dn)
# Up
IO_up = IOClass(NAMELIST_up)

In [ ]:
# Default
DATA_def = IO_def.create_data_file();
RESULTS_def = IO_def.create_result_file();
# Down
DATA_dn = IO_dn.create_data_file();
RESULTS_dn = IO_dn.create_result_file();
# Down
DATA_up = IO_up.create_data_file();
RESULTS_up = IO_up.create_result_file();

As in the previous sensitivity study we put everything in a list of lists

In [ ]:
# List of lists with our experiments
exp_list = [[NAMELIST_def, DATA_def, RESULTS_def, IO_def],
            [NAMELIST_dn, DATA_dn, RESULTS_dn, IO_dn],
            [NAMELIST_up, DATA_up, RESULTS_up, IO_up]
           ]

And lets runs the experiments

In [ ]:
# We are still not doing any evaluation
stakes_loc = None
df_stakes_data = None
stake_names = None

In [ ]:
for exp in exp_list:
    
    # We pass the index of our point to cosipy_core, since python is zero
    # indexed we have to subtract one.
    x = 0
    y = 0
    # DATA is now exp[1] and the namelist exp[0]
    model = cosipy_core(exp[1].isel(lat=y, lon=x), y, x, exp[0],
                        stake_names=stake_names,
                        stake_data=df_stakes_data)
    # The corresponding IO instance is exp[3]
    # Create numpy arrays which aggregates all local results
    exp[3].create_global_result_arrays()


    # Here we are unpacking the results from the model run,
    # getting ready to save it to our RESULTS dataframe.
    indY, indX, local_restart, RAIN, SNOWFALL, LWin, LWout, H, LE, B,\
    QRR, MB, surfMB, Q, SNOWHEIGHT, TOTALHEIGHT, TS, ALBEDO, NLAYERS,\
    ME, intMB, EVAPORATION, SUBLIMATION, CONDENSATION, DEPOSITION,\
    REFREEZE, subM, Z0, surfM, MOL, LAYER_HEIGHT, LAYER_RHO, LAYER_T,\
    LAYER_LWC, LAYER_CC, LAYER_POROSITY, LAYER_ICE_FRACTION,\
    LAYER_IRREDUCIBLE_WATER, LAYER_REFREEZE, stake_names, stat,df_eval = model
    
    
                   
    exp[3].copy_local_to_global(indY, indX, RAIN, SNOWFALL, LWin, LWout, H, LE,
                                B, QRR, MB, surfMB, Q, SNOWHEIGHT, TOTALHEIGHT,
                                TS, ALBEDO, NLAYERS, ME, intMB, EVAPORATION, 
                                SUBLIMATION,  CONDENSATION, DEPOSITION,
                                REFREEZE, subM, Z0, surfM, MOL, LAYER_HEIGHT,
                                LAYER_RHO, LAYER_T, LAYER_LWC, LAYER_CC,
                                LAYER_POROSITY, LAYER_ICE_FRACTION,
                                LAYER_IRREDUCIBLE_WATER,
                                LAYER_REFREEZE)

    # Write results to file
    exp[3].write_results_to_file()
    
print('Finished!')

And we can re-use the snippet of code to plot the results

In [ ]:
labels = ['Default', '-2$\degree$C', '+2$\degree$C']
fig, ax = plt.subplots(figsize=(10, 5))
for exp, label in zip(exp_list, labels):
    # Get the data and plot it RESULTS are kept at the third spot, index 2
    exp[2].surfMB.plot(ax=ax, label=label)
plt.legend(); 

<div class="alert alert-warning">
    <details>
        <summary>
            <b>Question: Do you understand the plot above?</b> <i>Click me for an explanation</i>
        </summary>
        Similarly to the albedo experiment we did in an earlier notebook changing the temperature directly affects the surface mass balance. Lowering the temperature (negative bias) leads to less mass loss during the day, while increasing the temperature results in increased mass loss.
    </details>
</div>

## Next steps
[Back to overview](welcome.ipynb)
